<a href="https://colab.research.google.com/github/vvoroby/semestr_4_numerical_methods/blob/main/%D0%9A%D1%83%D1%80%D1%81%D0%BE%D0%B2%D0%B0%D1%8F_%22%D0%A7%D0%B8%D1%81%D0%BB%D0%B5%D0%BD%D0%BD%D1%8B%D0%B5_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D1%8B_%D1%80%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D1%8F_%D0%BF%D1%80%D0%BE%D0%B1%D0%BB%D0%B5%D0%BC%D1%8B_%D1%81%D0%BE%D0%B1%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D1%8B%D1%85_%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D0%B9%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Численные методы решения проблемы собственных значений.**

In [ ]:
import math
import cmath
import numpy as np

## **1. Степенной метод (степенные итерации).**
Степенной метод предназначен для решения частичной проблемы собственных значений — нахождения доминирующих собственного значения и собственного вектора матрицы.

In [ ]:
def power_method(B, tol, x): 
  A = B.copy()
  iter = 0
  x = x.dot(1/np.linalg.norm(x, ord = 2)) 
  lamda = 0
  lamda_next = 1
  while abs(lamda_next - lamda) > tol :
    y = A.dot(x) 
    lamda = lamda_next 
    lamda_next = (y.dot(x))/np.linalg.norm(x, ord = 2)
    x = y.dot(1/np.linalg.norm(y, ord = 2))
    iter = iter + 1
  print ('Кол-во итераций в степенном методе: ', iter)
  return round(lamda_next, 7), x 


In [ ]:
def power_method_iter(B, tol, x): 
  A = B.copy()
  iter = 0
  x = x.dot(1/np.linalg.norm(x, ord = 2)) 
  lamda = 0
  lamda_next = 1
  while abs(lamda_next - lamda) > tol :
    if iter == 1:
      print ('1 - ', round(lamda_next, 7))
    if iter == 3:
      print ('3 - ', round(lamda_next, 7))
    if iter == 5:
      print ('5 - ', round(lamda_next, 7))
    if iter%10 == 0:
      print (iter,' - ', round(lamda_next, 7))

    y = A.dot(x) 
    lamda = lamda_next 
    lamda_next = (y.dot(x))/np.linalg.norm(x, ord = 2) 
    x = y.dot(1/np.linalg.norm(y, ord = 2))
    iter = iter + 1
  print ('Кол-во итераций в степенном методе: ', iter)
  return round(lamda_next, 7), x 

## **2. Обратные степенные итерации.**

In [ ]:
def inverse_power_iterations(B, tol, x):  
  A = B.copy()
  iter = 0
  x = x.dot(1/np.linalg.norm(x, ord = 2)) 
  lamda = 0
  lamda_next = 1
  while abs(lamda_next - lamda) > tol :
    y = np.linalg.solve(np.array(A), np.array(x))
    lamda = lamda_next 
    lamda_next = ((x.dot(y))/np.linalg.norm(y, ord = 2))/((y.dot(y))/np.linalg.norm(y, ord = 2)) 
    x = y.dot(1/np.linalg.norm(y, ord = 2))
    iter = iter + 1
  print ('Кол-во итераций в обратном степенном методе: ', iter)
  return round(lamda_next, 7), x 

## **3. Базовый QR - алгоритм.**

In [ ]:
def convergence(A):
    sum = 0
    el = np.diag(A, k=-1) 
    for i in range(0, len(el)-1):
      sum += abs(el[i]*el[i+1])
    return(sum)

def QR_algorithm(B, tol):   
  A = B.copy()
  iter = 0
  while convergence(A) > tol:  
    Q, R = np.linalg.qr(A)
    A = np.round(R.dot(Q),5)
    iter += 1
  print ('Кол-во итераций в базовом QR алгоритме: ', iter)
  return A 

## **3.1. Модифицированный QR-алгоритм со сдвигами**

In [ ]:
def convergence(A):
    sum = 0
    el = np.diag(A, k=-1) 
    for i in range(0, len(el)-1):
      sum += abs(el[i]*el[i+1])
    return(sum)
    
def modified_QR_algorithm(B, tol):
  A = B.copy()
  iter = 0
  shift = 0 
  I = np.eye(len(A)) 
  while convergence(A) > tol:   
    vI = I.dot(shift)
    Q, R = np.linalg.qr(A - vI)
    A = np.round((R.dot(Q) + vI),5)
    shift = A[0][0]
    iter += 1
  print ('Кол-во итераций в QR алгоритме со сдвигом: ', iter)
  return A

## **4. Метод Якоби поиска собственных чисел.**

Идея метода Якоби состоит в том, чтобы подходящими преобразованиями подобия от шага к шагу уменьшать норму внедиагональной части матрицы. 

In [ ]:
def frobenius_norm(A): 
  sum = 0
  for i in range(len(A)):
    for j in range(len(A)):
      if i!=j:
        sum += A[i][j]*A[i][j]
  return math.sqrt(sum)
 
def Elem_max(a): 
  n = len(a)
  Max = 0.0
  for i in range(n-1):
    for j in range(i+1,n):
      if abs(a[i][j]) >= Max:
        Max = abs(a[i][j])
        k = i
        l = j
  return k, l

def rotation_matrix(A, i, j):
  n = len(A)
  q = A[i][i] - A[j][j]
  if q == 0:
    c = (math.sqrt(2))/2
    s = (math.sqrt(2))/2
  else:
    p = 2*A[i][j]
    d = math.sqrt(p*p + q*q)
    r = abs(q)/(2*d)
    c = math.sqrt(0.5 + r)
    s = (math.sqrt(0.5 - r))*(np.sign(p*q))

  I = np.eye(n) 
  I[i][j] = -s
  I[j][i] = s
  I[i][i] = c
  I[j][j] = c
  return(I)
    
def jacobi_method_max(B, tol):
  A = B.copy()
  iter = 0
  n = len(A)
  vec = np.eye(n)
  while frobenius_norm(A) >= tol: 
    i, j = Elem_max(A) 
    Q = rotation_matrix(A, i, j)
    A = np.round(np.transpose(Q).dot(A).dot(Q),2) 
    vec = vec.dot(Q)
    iter += 1

  print ('Кол-во итераций в методе Якоби для поиска собственных чисел c обнулением максимального: ', iter)
  return np.diag(A), vec 

In [ ]:
def frobenius_norm(A): 
  sum = 0
  for i in range(len(A)):
    for j in range(len(A)):
      if i!=j:
        sum += A[i][j]*A[i][j]
  return math.sqrt(sum)

def Elem_not_zero(a): 
  n = len(a)
  for i in range(n-1):
    for j in range(i+1,n):
      if abs(a[i][j]) > 0.000001:
        Max = abs(a[i][j])
        k = i 
        l = j
  return k, l

def rotation_matrix(A, i, j):
  n = len(A)
  q = A[i][i] - A[j][j]
  if q == 0:
    c = (math.sqrt(2))/2
    s = (math.sqrt(2))/2
  else:
    p = 2*A[i][j]
    d = math.sqrt(p*p + q*q)
    r = abs(q)/(2*d)
    c = math.sqrt(0.5 + r)
    s = (math.sqrt(0.5 - r))*(np.sign(p*q))

  I = np.eye(n) 
  I[i][j] = -s
  I[j][i] = s
  I[i][i] = c
  I[j][j] = c
  return(I)
    
def jacobi_method_zero(B, tol):
  A = B.copy()
  iter = 0
  n = len(A)
  vec = np.eye(n)
  while frobenius_norm(A) >= tol: 
    i, j = Elem_max(A) 
    Q = rotation_matrix(A, i, j)
    A = np.round(np.transpose(Q).dot(A).dot(Q),2) 
    vec = vec.dot(Q)
    iter += 1

  print ('Кол-во итераций в методе Якоби для поиска собственных чисел c обнулением не нуля: ', iter)
  return np.diag(A), vec 

## **5. Метод Якоби поиска сингулярных чисел.**

In [ ]:
def frobenius_norm(A): 
    sum = 0
    for i in range(len(A)):
      for j in range(len(A)):
        if i!=j:
          sum += A[i][j]*A[i][j]
    return math.sqrt(sum)
    
def Elem_not_zero(a): 
  n = len(a)
  for i in range(n-1):
    for j in range(i+1,n):
      if abs(a[i][j]) > 0.000001:
        Max = abs(a[i][j])
        k = i 
        l = j
  return k, l
    
def rotation_matrix(A, i, j):
  n = len(A)
  q = A[i][i] - A[j][j]
  if q == 0:
    c = (math.sqrt(2))/2
    s = (math.sqrt(2))/2
  else:
    p = 2*A[i][j]
    d = math.sqrt(p*p + q*q)
    r = abs(q)/(2*d)
    c = math.sqrt(0.5 + r)
    s = (math.sqrt(0.5 - r))*(np.sign(p*q))

  I = np.eye(n) 
  I[i][j] = -s
  I[j][i] = s
  I[i][i] = c
  I[j][j] = c
  return(I)

def search_singular_values(B, tol):
  G = B.copy()
  A = givens_method(np.transpose(G).dot(G))
  iter = 0
  n = len(A)
  vec = np.eye(n)
    
  while frobenius_norm(A) >= tol: 
    i, j = Elem_not_zero(A) 
    Q = rotation_matrix(A, i, j)

    A = np.round(np.transpose(Q).dot(A).dot(Q),5)
    iter += 1
    vec = vec.dot(Q)

  print ('Кол-во итераций в методе Якоби для поиска сингулярных чисел: ', iter)
  singular = np.array([[0]*n]*n)
  for i in range(len(A)):
    for j in range(len(A)):
      if i == j:
        singular[i][j] = math.sqrt(A[i][j])
      
  return singular, vec

### Метод Гивенса
Метод Гивенса приводит любую симметричную (несимметричную) матрицу к подобной ей трехдиагональной (почти треугольной) матрице и основан на подобных преобразованиях исходной матрицы с помощью матриц вращения.

In [ ]:
def givens_method(m):
  A = m.copy()
  n = len(A)
  for i in range(n-1):
    for j in range(i, n):
      t = np.eye(n)
      c = (A[i-1][i])/(math.sqrt((A[i-1][i])**2+(A[i-1][j])**2))
      s = -(A[i-1][j])/(math.sqrt((A[i-1][i])**2+(A[i-1][j])**2))
      t[i][i] = c
      t[j][j] = c
      t[i][j] = s
      t[j][i] = -s
      A = np.round((np.transpose(t).dot(A)).dot(t),3)
  return A


# **ПРИМЕРЫ**

### Пример 1
Применение степенного метода и метода обратных итераций.

In [ ]:
m = np.array([[1, 2, 0, 0], [4, -6 , 280, 8], [-14, 2 , -76 , 10], [5,  9 , -9, -12]])
x = np.array([1,1,1,1])
tol=0.00001
print(m)
power_method(m,tol,x)
inverse_power_iterations(m,tol,x)

[[  1   2   0   0]
 [  4  -6 280   8]
 [-14   2 -76  10]
 [  5   9  -9 -12]]
Кол-во итераций в степенном методе:  16
Кол-во итераций в обратном степенном методе:  17


(3.9883022, array([0.46850826, 0.7000215 , 0.00288013, 0.53894491]))

### Пример 2
Применение степенного метода и метода обратных итераций к жордановой клетке.

In [ ]:
m = np.array([[1,1,0,0],[0,1,1,0],[0,0,1,1],[0,0,0,1]])
tol=0.00001
print(m)
power_method(m,tol,x)
inverse_power_iterations(m,tol,x)

[[1 1 0 0]
 [0 1 1 0]
 [0 0 1 1]
 [0 0 0 1]]
Кол-во итераций в степенном методе:  551
Кол-во итераций в обратном степенном методе:  1


(1.0, array([0.        , 0.70710678, 0.        , 0.70710678]))

### Пример 3
Применение степенного метода и метода обратных итераций к матрице Уилкинсона.

In [ ]:
def generate_wilkinson_matrix(n):
  m = np.zeros((n,n))
  el = n
  for i in range(n):
    for j in range(n):
      if i == j:
        m[i][j] = el
        el -= 1
      if i == j-1:
        m[i][j]= n
  return m

tol=0.00001
wilkinson_matrix = generate_wilkinson_matrix(20)
power_method(wilkinson_matrix, tol, np.array([1]*len(wilkinson_matrix)))
inverse_power_iterations(wilkinson_matrix, tol, np.array([1]*len(wilkinson_matrix)))

Кол-во итераций в степенном методе:  223
Кол-во итераций в обратном степенном методе:  21


(0.9999915,
 array([-5.10165497e-01,  4.84657464e-01, -4.36191936e-01,  3.70763330e-01,
        -2.96610812e-01,  2.22458220e-01, -1.55720832e-01,  1.01218591e-01,
        -6.07311851e-02,  3.34021685e-02, -1.67010926e-02,  7.51549543e-03,
        -3.00619967e-03,  1.05217041e-03, -3.15651281e-04,  7.89128598e-05,
        -1.57825799e-05,  2.36738816e-06, -2.36738935e-07,  1.18369527e-08]))

### Пример 4 
Сдвиг спектра для матрицы с близкими по значению собственными числами.

In [ ]:
m_3 = np.array([[-2,3,4],[5,7,2],[60,3,5]])
lamda, vec = power_method_iter(m_3, tol, np.array([1]*len(m_3)))
print('Доминирующее собственное значение: ', lamda)

0  -  1
1 -  29.0
3 -  27.8447651
5 -  23.8872789
10  -  18.2558089
20  -  19.1815624
30  -  19.230119
40  -  19.2326253
50  -  19.2327546
Кол-во итераций в степенном методе:  52
Доминирующее собственное значение:  19.2327578


In [ ]:
m_3 = m_3 + 5*np.eye(len(m_3)) #делаем сдвиг
lamda, vec = power_method_iter(m_3, tol, np.array([1]*len(m_3)))
print('Доминирующее собственное значение: ', lamda - 5)

0  -  1
1 -  34.0
3 -  26.5643365
5 -  24.5717084
10  -  24.230511
Кол-во итераций в степенном методе:  18
Доминирующее собственное значение:  19.2327607


### Пример 5
Сравнение работы QR-алгоритмов (таблица 1).

In [ ]:
m = np.array([[1, 2, 0, 0], [4, -6 , 280, 8], [-14, 2 , -76 , 10], [5, 
  9 , -9, -12]])
print(m)

[[  1   2   0   0]
 [  4  -6 280   8]
 [-14   2 -76  10]
 [  5   9  -9 -12]]


In [ ]:
m_1 = givens_method(m)
print(m_1)

[[-102.46   155.173    0.       0.   ]
 [ -22.069    5.411    6.093    0.   ]
 [ -93.266  161.682    3.45    -0.235]
 [ -37.197   71.515    1.923    0.293]]


In [ ]:
QR_algorithm(m, 0.001)

Кол-во итераций в базовом QR алгоритме:  8


array([[-7.8681420e+01,  1.1741104e+02,  2.3646486e+02,  8.3893910e+01],
       [-9.0100000e-03, -2.7668210e+01, -3.0963280e+01,  1.4307800e+00],
       [ 1.0000000e-05,  1.1830000e-02,  9.3664300e+00,  1.2968000e+01],
       [ 0.0000000e+00, -0.0000000e+00, -2.1100000e-03,  3.9832100e+00]])

In [ ]:
QR_algorithm(m_1, 0.001)

Кол-во итераций в базовом QR алгоритме:  6


array([[-6.4154270e+01,  1.3452788e+02, -2.2461321e+02, -2.7800460e+01],
       [-8.6750000e-02, -3.8552190e+01,  3.8213830e+01, -1.2712000e-01],
       [ 1.6000000e-04,  1.0750000e-02,  8.9437000e+00,  4.3875700e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  4.5675000e-01]])

In [ ]:
modified_QR_algorithm(m, 0.001)

Кол-во итераций в QR алгоритме со сдвигом:  41


array([[-1.1681564e+02,  5.2236380e+01, -1.6776726e+02,  1.4186655e+02],
       [-6.5107960e+01,  1.0475870e+01, -8.6383780e+01,  9.0548010e+01],
       [-1.0000000e-05,  1.0000000e-05,  3.9882900e+00, -1.2970560e+01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  9.3514500e+00]])

In [ ]:
modified_QR_algorithm(m_1, 0.001)

Кол-во итераций в QR алгоритме со сдвигом:  48


array([[-1.0916312e+02, -1.0399904e+02,  2.1259106e+02,  6.6417590e+01],
       [ 3.0667930e+01,  6.4482300e+00, -5.1160180e+01, -2.1114600e+01],
       [-1.0000000e-05, -1.0000000e-05,  7.8479800e+00,  5.7955900e+00],
       [ 0.0000000e+00, -0.0000000e+00,  1.4080700e+00,  1.5608600e+00]])

### Пример 6
Сравнение QR-алгоритмов для матрицы с близкими по абсолютному значению числами (таблица 2)

In [ ]:
m = np.array([[4.07634, 1.02546, 1.05139, 1.25927],
              [-0.247737, 3.93355, 0.0104355, -0.0234149],
              [0.0389078, 0.0104355, -0.173069, -1.85544], 
              [-0.0873002, -0.0234149, -1.85544, 3.16318]]) #матрица с близкими по знач соьственными значениями
print(m)

[[ 4.07634    1.02546    1.05139    1.25927  ]
 [-0.247737   3.93355    0.0104355 -0.0234149]
 [ 0.0389078  0.0104355 -0.173069  -1.85544  ]
 [-0.0873002 -0.0234149 -1.85544    3.16318  ]]


In [ ]:
m_1 = givens_method(m)
print(m_1)

[[ 3.922  0.534  0.     0.   ]
 [ 0.519 -0.988  0.337 -0.   ]
 [-0.362 -0.521  2.765  0.132]
 [ 0.209  0.473 -0.355  0.741]]


In [ ]:
QR_algorithm(m,0.001)

Кол-во итераций в базовом QR алгоритме:  46


array([[ 4.00879e+00,  1.10812e+00, -5.99610e-01, -1.46927e+00],
       [-2.51310e-01,  3.98662e+00, -1.80000e-02, -1.00430e-01],
       [ 6.47700e-02,  3.45000e-03,  4.00462e+00,  2.58700e-02],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00, -9.99990e-01]])

In [ ]:
modified_QR_algorithm(m, 0.001)

Кол-во итераций в QR алгоритме со сдвигом:  2


array([[ 3.98600e+00,  2.59550e-01, -9.59400e-02, -2.03800e-02],
       [-1.22255e+00,  4.01550e+00,  1.47198e+00,  2.93310e-01],
       [-0.00000e+00, -2.00000e-05, -9.99800e-01,  3.61800e-02],
       [ 6.00000e-03, -8.00000e-05,  2.89700e-02,  3.99830e+00]])

In [ ]:
QR_algorithm(m_1,0.001)

Кол-во итераций в базовом QR алгоритме:  8


array([[ 3.97258e+00, -4.44000e-01,  3.16340e-01, -2.72370e-01],
       [ 1.82400e-02,  2.70856e+00,  8.81890e-01, -2.49920e-01],
       [-4.00000e-05, -1.07800e-02, -1.00868e+00, -3.09370e-01],
       [ 0.00000e+00,  1.40000e-04,  4.37400e-02,  7.67540e-01]])

In [ ]:
modified_QR_algorithm(m_1,0.001)

Кол-во итераций в QR алгоритме со сдвигом:  17


array([[ 1.53938e+00, -2.70801e+00,  3.19740e-01,  4.64000e-03],
       [-2.27438e+00,  1.42840e+00,  9.13700e-01, -3.95930e-01],
       [ 1.60000e-04, -2.20000e-04,  2.71237e+00,  2.99680e-01],
       [ 0.00000e+00,  0.00000e+00, -1.00000e-05,  7.59870e-01]])

### Пример 7
Примение нормы Фробениуса

In [ ]:
def frobenius_norm(A): # считает норму Фробениуса из внедиагональных элемнтов, используется для условия сходимости
  sum = 0
  for i in range(len(A)):
    for j in range(len(A)):
      if i!=j:
        sum += A[i][j]*A[i][j]
  return math.sqrt(sum)

In [ ]:
m = np.array([[69,0,0,0],
              [0,-60,0,0],
              [0,0,1,0],
              [0,0,0,100]])
print("Матрица A:")
print(m)
print("Норма Фробениуса для матрицы А: ", frobenius_norm(m))
m1 = np.array([[69,0,0.0001,0],
              [0,-60,0,0.0001],
              [0.0001,0,1,0],
              [0.0001,0,0,100]])
print("Матрица B:")
print(np.round(m1,4))
print("Норма Фробениуса для матрицы B: ", round(frobenius_norm(m1),7))

Матрица A:
[[ 69   0   0   0]
 [  0 -60   0   0]
 [  0   0   1   0]
 [  0   0   0 100]]
Норма Фробениуса для матрицы А:  0.0
Матрица B:
[[ 6.9e+01  0.0e+00  1.0e-04  0.0e+00]
 [ 0.0e+00 -6.0e+01  0.0e+00  1.0e-04]
 [ 1.0e-04  0.0e+00  1.0e+00  0.0e+00]
 [ 1.0e-04  0.0e+00  0.0e+00  1.0e+02]]
Норма Фробениуса для матрицы B:  0.0002


In [ ]:
frobenius_norm(m)

0.0

In [ ]:
m = np.array([[69,0,0.0009,0],
              [0,-60,0,0],
              [0,0,1,0],
              [0.0002,0,0,100]])
m

array([[ 6.9e+01,  0.0e+00,  9.0e-04,  0.0e+00],
       [ 0.0e+00, -6.0e+01,  0.0e+00,  0.0e+00],
       [ 0.0e+00,  0.0e+00,  1.0e+00,  0.0e+00],
       [ 2.0e-04,  0.0e+00,  0.0e+00,  1.0e+02]])

In [ ]:
frobenius_norm(m)

0.0009219544457292888

### Пример 8
Применение QR-алгоритма к симметричной матрице.

In [ ]:
m = np.array([[1,4,-7,5],
              [4,-6,2,8],
              [-7,2,1,1],
              [5,8,1,-1]])
print(m)

[[ 1  4 -7  5]
 [ 4 -6  2  8]
 [-7  2  1  1]
 [ 5  8  1 -1]]


In [ ]:
m_1 = givens_method(m)
print(m_1)

[[-4.786  8.767  0.     0.   ]
 [ 8.767  0.848  1.009  0.   ]
 [ 0.     1.009 -2.721 -2.001]
 [ 0.     0.    -2.001  1.589]]


In [ ]:
QR_algorithm(m, 0.001)

Кол-во итераций в базовом QR алгоритме:  42


array([[-1.207685e+01,  5.442300e-01, -4.000000e-05,  0.000000e+00],
       [ 5.442300e-01,  1.064246e+01, -1.480000e-03, -0.000000e+00],
       [-4.000000e-05, -1.480000e-03, -8.525860e+00,  1.000000e-05],
       [ 0.000000e+00,  0.000000e+00,  1.000000e-05,  4.960230e+00]])

In [ ]:
QR_algorithm(m_1, 0.001)

Кол-во итераций в базовом QR алгоритме:  9


array([[-1.121642e+01, -2.849100e-01, -0.000000e+00, -0.000000e+00],
       [-2.849100e-01,  7.306470e+00, -1.060000e-03, -0.000000e+00],
       [ 0.000000e+00, -1.060000e-03, -3.519630e+00, -6.272000e-02],
       [ 0.000000e+00,  0.000000e+00, -6.272000e-02,  2.359610e+00]])

In [ ]:
modified_QR_algorithm(m, 0.001)

Кол-во итераций в QR алгоритме со сдвигом:  35


array([[ 1.09976e+00, -1.12266e+01, -8.00000e-05,  0.00000e+00],
       [-1.12266e+01, -2.53412e+00,  8.00000e-05,  0.00000e+00],
       [-8.00000e-05,  8.00000e-05, -8.52589e+00,  0.00000e+00],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  4.96022e+00]])

In [ ]:
modified_QR_algorithm(m_1, 0.001)

Кол-во итераций в QR алгоритме со сдвигом:  34


array([[ 3.88507e+00,  7.19367e+00,  0.00000e+00, -0.00000e+00],
       [ 7.19367e+00, -7.79502e+00,  1.00000e-04,  0.00000e+00],
       [ 0.00000e+00,  1.00000e-04, -3.52031e+00,  5.00000e-05],
       [ 0.00000e+00,  0.00000e+00,  5.00000e-05,  2.36029e+00]])

### Приимер 9
Применение QR-алгоритма и метода Якоби к симметричной матрице(таблица 3).

In [ ]:
m = np.array([[-1,0,9,3,6],
              [0,2,10,4,-70],
              [9,10,-7,1,8],
              [3,4,1,-8,-56],
              [6,-70,8,-56,-23]])
m

array([[ -1,   0,   9,   3,   6],
       [  0,   2,  10,   4, -70],
       [  9,  10,  -7,   1,   8],
       [  3,   4,   1,  -8, -56],
       [  6, -70,   8, -56, -23]])

In [ ]:
m_1 = givens_method(m)
m_1

array([[-44.253,  85.173,   0.   ,   0.   ,   0.   ],
       [ 85.173,  21.851,   0.484,   0.   ,   0.   ],
       [  0.   ,   0.484,  -9.906,   3.76 ,  -0.   ],
       [  0.   ,   0.   ,   3.76 ,  -1.084,  -1.785],
       [  0.   ,   0.   ,  -0.   ,  -1.785,  -0.455]])

In [ ]:
tol = 0.00001

In [ ]:
QR_algorithm(m, tol)
QR_algorithm(m_1, tol)

Кол-во итераций в базовом QR алгоритме:  14
Кол-во итераций в базовом QR алгоритме:  8


array([[-100.91726,   17.26318,    0.     ,    0.     ,    0.     ],
       [  17.26318,   78.51622,    0.     ,    0.     ,    0.     ],
       [   0.     ,    0.     ,  -11.32769,   -0.     ,    0.     ],
       [   0.     ,    0.     ,   -0.     ,   -0.61955,   -1.61634],
       [   0.     ,    0.     ,    0.     ,   -1.61634,    0.50128]])

In [ ]:
jacobi_method_max(m,tol)
jacobi_method_zero(m,tol)

Кол-во итераций в методе Якоби для поиска собственных чисел c обнулением максимального:  21
Кол-во итераций в методе Якоби для поиска собственных чисел c обнулением не нуля:  21


(array([   6.95,   80.22,  -17.02,   -4.48, -102.67]),
 array([[ 0.7546994 , -0.03070466, -0.48022236,  0.44349289, -0.04676083],
        [ 0.14687229,  0.61079203, -0.32346299, -0.50571165,  0.49496369],
        [ 0.63254618,  0.01174413,  0.68146631, -0.34973553, -0.1141759 ],
        [-0.05334473,  0.44296709,  0.44625756,  0.64940325,  0.42433961],
        [ 0.07679202, -0.65546362,  0.03487163, -0.05941598,  0.74814713]]))

In [ ]:
jacobi_method_max(m_1,tol)
jacobi_method_zero(m_1,tol)

Кол-во итераций в методе Якоби для поиска собственных чисел c обнулением максимального:  12
Кол-во итераций в методе Якоби для поиска собственных чисел c обнулением не нуля:  12


(array([-102.56,   80.16,  -11.32,    1.64,   -1.77]),
 array([[ 8.25147746e-01,  5.64895861e-01, -4.54566592e-03,
         -1.18673154e-03, -1.33849320e-03],
        [-5.64909617e-01,  8.25150219e-01, -1.78951370e-03,
         -7.21384922e-04, -7.19760162e-04],
        [ 2.88094463e-03,  4.43871912e-03,  9.33909515e-01,
          2.41179513e-01,  2.63851204e-01],
        [-1.44232342e-04,  2.82372712e-04, -3.52802042e-01,
          7.40832414e-01,  5.71574975e-01],
        [ 2.59393477e-05,  2.24731510e-05, -5.76183217e-02,
         -6.26895404e-01,  7.76969935e-01]]))

### Пример 10
Поиск сингулярных чисел и сингулярное раздожение.

In [ ]:
m = np.array([[-1,2,3,4],
              [2,-6,4,1],
              [3,4,-11,-12],
              [4,1,-12,-16]])
m

array([[ -1,   2,   3,   4],
       [  2,  -6,   4,   1],
       [  3,   4, -11, -12],
       [  4,   1, -12, -16]])

In [ ]:
c,v = search_singular_values(m, 0.0001)

Кол-во итераций в методе Якоби для поиска сингулярных чисел:  21
